In [1]:
import pandas as pd
toc_sub = pd.read_excel(r'/mnt/primus/xuemeng_tmp_harbour/soc_eu/foregs/subsoil/C_TOC_data_2v3_9Feb06.xls')
toc_top = pd.read_excel(r'/mnt/primus/xuemeng_tmp_harbour/soc_eu/foregs/topsoil/T_toc_data_2v3_8Feb06.xls')
toc_sub['hzn_top'] = 50
toc_sub['hzn_btm'] = 200
toc_top['hzn_top'] = 0
toc_top['hzn_btm'] = 25
toc = pd.concat([toc_top,toc_sub]).reset_index(drop = True)
toc = toc.drop(columns=['COUNTRY'])
toc['TOC'] = toc['TOC']*10

ph_sub = pd.read_excel(r'/mnt/primus/xuemeng_tmp_harbour/soc_eu/foregs/subsoil/C_pH_data_2v4_9Feb06.xls')
ph_top = pd.read_excel(r'/mnt/primus/xuemeng_tmp_harbour/soc_eu/foregs/topsoil/T_pH_data_2v4_8Feb06.xls')
ph_sub['hzn_top'] = 50
ph_sub['hzn_btm'] = 200
ph_top['hzn_top'] = 0
ph_top['hzn_btm'] = 25
ph = pd.concat([ph_top,ph_sub]).reset_index(drop = True)

chemical_sub = pd.read_excel(r'/mnt/primus/xuemeng_tmp_harbour/soc_eu/foregs/subsoil/C_XRF_data_2v6_8Feb06.xls')
chemical_top = pd.read_excel(r'/mnt/primus/xuemeng_tmp_harbour/soc_eu/foregs/topsoil/T_XRF_data_2v6_8Feb06.xls')
chemical_sub['hzn_top'] = 50
chemical_sub['hzn_btm'] = 200
chemical_top['hzn_top'] = 0
chemical_top['hzn_btm'] = 25
chemical = pd.concat([chemical_top,chemical_sub]).reset_index(drop = True)
chemical = chemical.dropna(subset=['LONG', 'LAT', 'hzn_top', 'hzn_btm'], how = 'any')

chemical['K'] = chemical['K2O_T']*0.8301*10000 # 0.8301*K2O = K, % -> mg/kg
chemical['P'] = chemical['P2O5_T']*0.4364*10000 # 0.4364*P2O5 = P, % -> mg/kg
chemical = chemical.drop(columns=['SIO2_T', 'TIO2_T', 'AL2O3_T',
       'FE2O3_T', 'MNO_T', 'MGO_T', 'CAO_T', 'NA2O_T',
       'BA_T', 'CR_T', 'RB_T', 'SN_T', 'SR_T', 'W_T', 'Y_T', 'ZN_T', 'ZR_T',
       'SIO2_SI', 'TIO2_SI', 'AL2O3_SI',
       'FE2O3_SI', 'MNO_SI', 'MGO_SI', 'CAO_SI', 'NA2O_SI', 'K2O_SI',
       'P2O5_SI', 'BA_SI', 'CR_SI', 'RB_SI', 'SN_SI', 'SR_SI', 'W_SI', 'Y_SI',
       'ZN_SI', 'ZR_SI','COUNTRY','K2O_T','P2O5_T'])
chemical = chemical.rename(columns = {'GTN':'gtn','LONG':'long','LAT':'lat','CNTRY':'cntry'})

# merge
toc = toc.rename(columns = {'GTN':'gtn','LONG':'long','LAT':'lat','CNTRY':'cntry'})
foregs = toc.merge(ph, on = ['gtn', 'long', 'lat', 'cntry', 'hzn_top', 'hzn_btm'], how='outer')
foregs = foregs.merge(chemical, on = ['gtn', 'long', 'lat', 'cntry', 'hzn_top', 'hzn_btm'], how='outer')
foregs = foregs.dropna(subset=['long', 'lat', 'hzn_top', 'hzn_btm'], how = 'any')
foregs = foregs.dropna(subset=['TOC','pH','K','P'], how = 'all')
foregs.loc[foregs['cntry']=='GR','cntry'] = 'EL'
foregs.loc[foregs['cntry']=='SL','cntry'] = 'SI'
foregs = foregs.rename(columns={'TOC':'oc','pH':'ph_cacl2','long':'lon','cntry':'nuts0'})
foregs = foregs.drop(columns=['cnt'])


In [2]:
time_periods = {
    'AL': 'nan',
    'AT': '1998',
    'BE': '1998',
    'HR': '1998,1999',
    'CZ': '1998',
    'DK': '1999',
    'EE': '1999',
    'FI': '1997,1998',
    'FR': '1998,1999',
    'DE': '2001',
    'EL': '2000',
    'HU': '1999',
    'IE': '1998',
    'IT': '1998,2001',
    'LV': '1998',
    'LT': '1997',
    'NL': '2000',
    'NO': '1997',
    'PL': '1999',
    'PT': '1998,1999,2000',
    'SK': '1998',
    'SI': '2001',
    'ES': '2000,2001',
    'SE': '1998,1999',
    'CH': '1998',
    'UK': '1998'
}

foregs['time_real'] = 0
for i in time_periods:
    foregs.loc[foregs['nuts0']==i,'time_real'] = time_periods.get(i)
    
foregs['time'] = 2000

In [3]:
foregs.to_csv('/mnt/primus/xuemeng_tmp_harbour/soc_eu/data/foregs_harmonized_v1.csv',index=False)

In [4]:
foregs

gtn        lon        lat    oc nuts0  hzn_top  hzn_btm  ph_cacl2  \
2     N19W10T1 -15.570000  27.960000  18.4    ES        0       25       NaN   
3     N24E14T1  23.830000  35.500000  19.1    EL        0       25       NaN   
4     N24E15T2  24.670000  35.400000  35.2    EL        0       25       NaN   
5     N24E15T4  25.280000  35.040000  15.4    EL        0       25       NaN   
6     N24E15T5  25.200000  35.270000  11.7    EL        0       25       NaN   
...        ...        ...        ...   ...   ...      ...      ...       ...   
2196  N38E11T5  26.554167  56.135833   NaN    ES        0       25       NaN   
2197  N39E11T3  25.806944  57.798889   NaN    ES        0       25       NaN   
2198  N39E11T4  26.425000  57.425556   NaN    ES        0       25       NaN   
2199  N39E11T5  25.412500  57.110000   NaN    ES        0       25       NaN   
2200  N43E04T4  11.947641  63.687675   NaN    NO        0       25       NaN   

              K        P  time_real  time  
2           NaN      NaN  2000,2001  2000  
3           NaN      NaN       2000  2000  
4           NaN      NaN       2000  2000  
5           NaN      NaN       2000  2000  
6           NaN      NaN       2000  2000  
...         ...      ...        ...   ...  
2196  21233.958  558.592  2000,2001  2000  
2197  21192.453  811.704  2000,2001  2000  
2198  13331.406  362.212  2000,2001  2000  
2199  30813.312  658.964  2000,2001  2000  
2200   1759.812  680.784       1997  2000  

[2197 rows x 12 columns]